# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import nltk

import pickle
import re

nltk.download(['wordnet', 'punkt', 'averaged_perceptron_tagger'])

In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('messages', con=engine)
# define the label columns
label_cols = ['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']
# seperate predictor and response variable
X = df['message']
y = df[label_cols].as_matrix()
df.shape

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app


(26216, 40)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize_word(text):
    # transform string to lowercase
    text = text.lower()
    # substitute any characters but letters and numbers
    text = re.sub(r"[^a-z0-9]", " ", text)
    # tokenize words
    token = word_tokenize(text)
    
    # reduce words to their stem
    token =  [PorterStemmer().stem(tok) for tok in token]
    
    # lemmatize the words
    token = [WordNetLemmatizer().lemmatize(tok) for tok in token]
    
    return token

def tokenize_punct(text):
    # transform string to lowercase
    text = text.lower()
    # substitute any characters but letters and numbers
    text = re.sub(r"[^a-z0-9!?]", " ", text)
    # tokenize words
    token = word_tokenize(text)
    
    # reduce words to their stem
    token =  [PorterStemmer().stem(tok) for tok in token]
    
    # lemmatize the words
    token = [WordNetLemmatizer().lemmatize(tok) for tok in token]
    
    return token

print(tokenize_punct(X[7]))

['plea', 'we', 'need', 'tent', 'and', 'water', 'we', 'are', 'in', 'silo', 'thank', 'you', '!']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# build a simple pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_word)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# perform test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# fit the pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
# predict the train data
y_pred_train = pipeline.predict(X_train)
# predict the test data
y_pred_test = pipeline.predict(X_test)

In [7]:
# score the train data
print(classification_report(y_train.astype(bool), y_pred_train.astype(bool), target_names=label_cols))

                        precision    recall  f1-score   support

               related       0.99      1.00      0.99     13496
               request       1.00      0.93      0.96      3002
                 offer       1.00      0.66      0.80        80
           aid_related       1.00      0.97      0.98      7315
          medical_help       1.00      0.85      0.92      1383
      medical_products       1.00      0.85      0.92       867
     search_and_rescue       1.00      0.76      0.86       498
              security       1.00      0.74      0.85       311
              military       1.00      0.87      0.93       593
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.92      0.96      1129
                  food       1.00      0.95      0.97      1958
               shelter       1.00      0.92      0.96      1539
              clothing       1.00      0.86      0.92       278
                 money       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [8]:
# score the test data
print(classification_report(y_test.astype(bool), y_pred_test.astype(bool), target_names=label_cols))

                        precision    recall  f1-score   support

               related       0.82      0.93      0.87      6598
               request       0.85      0.37      0.52      1472
                 offer       0.00      0.00      0.00        38
           aid_related       0.75      0.54      0.63      3545
          medical_help       0.56      0.06      0.10       701
      medical_products       0.72      0.06      0.11       446
     search_and_rescue       1.00      0.04      0.07       226
              security       0.00      0.00      0.00       160
              military       0.53      0.08      0.14       267
           child_alone       0.00      0.00      0.00         0
                 water       0.87      0.19      0.31       543
                  food       0.82      0.46      0.59       965
               shelter       0.81      0.19      0.31       775
              clothing       0.83      0.12      0.21       127
                 money       0.89      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = {
    'vect__tokenizer': (tokenize_word, tokenize_punct),
    'vect__ngram_range': ((1, 1), (1, 2)),
    'clf__estimator__n_estimators': (1, 20),
    'clf__estimator__min_samples_split': (2, 5),
}

cv = GridSearchCV(pipeline, param_grid=parameters)

# fit model to the train data
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__tokenizer': (<function tokenize_word at 0x7fb80918d268>, <function tokenize_punct at 0x7fb7ceb232f0>), 'vect__ngram_range': ((1, 1), (1, 2)), 'clf__estimator__n_estimators': (1, 20), 'clf__estimator__min_samples_split': (2, 5)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
print("Best Parameters:", cv.best_params_)

Best Parameters: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 20, 'vect__ngram_range': (1, 2), 'vect__tokenizer': <function tokenize_punct at 0x7fb7ceb232f0>}


In [11]:
# predict the labels
y_pred_train = cv.predict(X_train) 
y_pred_test = cv.predict(X_test)

In [12]:
# score the test data
print(classification_report(y_test.astype(bool), y_pred_test.astype(bool), target_names=label_cols))

                        precision    recall  f1-score   support

               related       0.81      0.95      0.88      6598
               request       0.88      0.40      0.55      1472
                 offer       0.00      0.00      0.00        38
           aid_related       0.75      0.55      0.63      3545
          medical_help       0.64      0.06      0.10       701
      medical_products       0.80      0.05      0.10       446
     search_and_rescue       0.81      0.06      0.11       226
              security       0.00      0.00      0.00       160
              military       0.36      0.01      0.03       267
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.22      0.35       543
                  food       0.86      0.42      0.57       965
               shelter       0.86      0.22      0.35       775
              clothing       0.73      0.06      0.12       127
                 money       0.89      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [62]:
# get the length of each message
class MessageLength(BaseEstimator, TransformerMixin):
    def msg_len(self, msg):
        return len(tokenize_punct(msg))

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X = pd.Series(X).apply(self.msg_len)
        return pd.DataFrame(X)
    
# extract the parts of speech in each message
def tokenize_pos(msg):
    tokens = tokenize_word(msg)
    try:
        pos_tags = list(zip(*nltk.pos_tag(tokens)))[1]
    except:
        pos_tags = 'INVALID'
        
    return list(pos_tags)

In [73]:
# build a pipeline
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize_punct)),
            ('tfidf', TfidfTransformer()),
            ])),
        ('pos_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize_pos)),
            ('tfidf', TfidfTransformer()),
            ])),
        ('msg_len', MessageLength()),
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, 
                                                        min_samples_split=2))),
])

# fit the pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [74]:
# predict the test data
y_pred_test = pipeline.predict(X_test)

In [75]:
print(classification_report(y_test > 0.5, y_pred_test > 0.5, target_names=label_cols))

                        precision    recall  f1-score   support

               related       0.80      0.97      0.88      6598
               request       0.91      0.37      0.53      1472
                 offer       0.00      0.00      0.00        38
           aid_related       0.78      0.59      0.67      3545
          medical_help       0.75      0.04      0.08       701
      medical_products       0.83      0.03      0.06       446
     search_and_rescue       0.75      0.01      0.03       226
              security       0.00      0.00      0.00       160
              military       0.38      0.02      0.04       267
           child_alone       0.00      0.00      0.00         0
                 water       0.93      0.17      0.29       543
                  food       0.90      0.27      0.42       965
               shelter       0.87      0.15      0.25       775
              clothing       1.00      0.04      0.08       127
                 money       0.83      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [76]:
filename = 'model.p'
with open(filename, 'wb') as file:
    pickle.dump(pipeline, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [78]:
def train(X, y, filename='model.p'):
    # build a simple pipeline
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize_punct)),
                ('tfidf', TfidfTransformer()),
                ])),
            ('pos_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize_pos)),
                ('tfidf', TfidfTransformer()),
                ])),
            ('msg_len', MessageLength()),
        ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])
    
    # get parameters for cross validation
    parameters = {
        'features__text_pipeline__vect__tokenizer': (tokenize_word, tokenize_punct),
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__n_estimators': (20, 50),
        'clf__estimator__min_samples_split': [2, 5],
    }
    
    # instantiate the cross validator
    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    # fit the model to the data
    cv.fit(X, y)
    
    # save the model
    with open(filename, 'wb') as file:
        pickle.dump(pipeline, file)